In [1]:
import pandas as pd
import numpy as np
from math import exp, log, sqrt, pi
from scipy.stats import norm

S_e_minus_qT = 2360.1743583417406
e_minus_rT = 0.993169427730553
S = 2381
T = 138/252

q = -log(S_e_minus_qT / S) / T
r = -log(e_minus_rT) / T

F = S_e_minus_qT * 1/e_minus_rT

df = pd.read_csv("prices.csv")

df["y"] = np.log(F/df["Strike"])
df["alpha_C"] = df["Mid_Call"]/(df["Strike"] * exp(-r*T))
df["R_C"] = 2*df["alpha_C"] - np.exp(df["y"]) + 1
df["B_C"] = 4 * (np.exp((2/pi)*df["y"]) + np.exp(-(2/pi)*df["y"])) - 2*np.exp(-df["y"])*(np.exp(
    (1-2/pi)*df["y"]) + np.exp(-(1-2/pi)*df["y"])) * (np.exp(2*df["y"]) + 1 - np.power(df["R_C"], 2))
df["C_C"] = np.exp(-2*df["y"]) * (np.power(df["R_C"], 2) - np.power(np.exp(
    df["y"]) - 1, 2)) * (np.power(np.exp(df["y"] + 1), 2) - np.power(df["R_C"], 2))

df["A"] = np.power(np.exp((1-2/pi)*df["y"]) - np.exp(-(1-2/pi)*df["y"]), 2)


def A(y: float) -> float:
    return (exp((1-2/pi)*y) - exp(-(1-2/pi)*y))**2


df["alpha_P"] = df["Mid_Put"]/(df["Strike"] * exp(-r*T))
df["R_P"] = 2*df["alpha_P"] + np.exp(df["y"]) - 1
df["B_P"] = 4*(np.exp((2/pi)*df["y"]) + np.exp(-(2/pi)*df["y"])) - 2*np.exp(-df["y"])*(np.exp((1-2/pi)
                                                                                              * df["y"]) + np.exp(-(1-2/pi)*df["y"])) * (np.exp(2*df["y"]) + 1 - np.power(df["R_P"], 2))
df["C_P"] = np.exp(-2*df["y"]) * (np.power(df["R_P"], 2) - np.power(np.exp(
    df["y"]) - 1, 2)) * (np.power(np.exp(df["y"] + 1), 2) - np.power(df["R_P"], 2))

df["beta_C"] = 2 * df["C_C"] / \
    (df["B_C"] + np.sqrt(np.power(df["B_C"], 2) + 4*df["A"]*df["C_C"]))
df["gamma_C"] = -pi/2*np.log(df["beta_C"])

df["beta_P"] = 2 * df["C_P"] / \
    (df["B_P"] + np.sqrt(np.power(df["B_P"], 2) + 4*df["A"]*df["C_P"]))
df["gamma_P"] = -pi/2*np.log(df["beta_P"])

df["CP_00"] = df["Strike"]*np.exp(-r*T)


def calculate_sigma_c(row):
    g = row["gamma_C"]
    y = row["y"]
    if y >= 0:
        C_0 = row["CP_00"] * (exp(y) * A(sqrt(2*y)) - 1/2)
        if row['Mid_Call'] <= C_0:
            return 1 / sqrt(T) * (sqrt(g+y) - sqrt(g-y))
        else:
            return 1 / sqrt(T) * (sqrt(g+y) + sqrt(g-y))
    else:
        C_0 = row["CP_00"] * (exp(y)/2 - A(-sqrt(-2*y)))
        if row['Mid_Call'] <= C_0:
            return 1 / sqrt(T) * (-sqrt(g+y) + sqrt(g-y))
        else:
            return 1 / sqrt(T) * (sqrt(g+y) + sqrt(g-y))


def calculate_sigma_p(row):
    g = row["gamma_P"]
    y = row["y"]
    if y >= 0:
        P_0 = row["CP_00"] * (1/2 - exp(y) * A(-sqrt(2*y)))
        if row['Mid_Call'] <= P_0:
            return 1 / sqrt(T) * (sqrt(g+y) - sqrt(g-y))
        else:
            return 1 / sqrt(T) * (sqrt(g+y) + sqrt(g-y))
    else:
        P_0 = row["CP_00"] * (A(sqrt(-2*y)) - exp(y)/2)
        if row['Mid_Call'] <= P_0:
            return 1 / sqrt(T) * (-sqrt(g+y) + sqrt(g-y))
        else:
            return 1 / sqrt(T) * (sqrt(g+y) + sqrt(g-y))

# Apply calculation to DataFrame
# df['sigma_C'] = df.apply(calculate_sigma_c, axis=1)
# df['sigma_P'] = df.apply(calculate_sigma_p, axis=1)


df


,Strike,Mid_Call,Mid_Put,y,alpha_C,R_C,B_C,C_C,A,alpha_P,R_P,B_P,C_P,beta_C,gamma_C,beta_P,gamma_P,CP_00
0,2150,260.000000,35.250000,0.100122,0.121762,0.138218,0.039986,0.059099,5.297004e-03,0.016508,0.138322,0.040090,0.059309,1.265745,-0.370175,1.267228,-0.372015,2135.314270
1,2175,238.850006,38.950001,0.088561,0.110571,0.128542,0.037688,0.058619,4.143964e-03,0.018031,0.128663,0.037802,0.058849,1.353841,-0.475866,1.355372,-0.477641,2160.143505
2,2200,218.149994,43.000000,0.077132,0.099841,0.119497,0.035585,0.057908,3.143164e-03,0.019680,0.119545,0.035627,0.057991,1.443325,-0.576403,1.443836,-0.576959,2184.972741
3,2225,197.949997,47.599998,0.065833,0.089578,0.111108,0.033638,0.056920,2.289535e-03,0.021540,0.111129,0.033655,0.056953,1.532309,-0.670378,1.532489,-0.670563,2209.801977
4,2250,178.149994,52.650000,0.054659,0.079722,0.103264,0.031703,0.055399,1.578220e-03,0.023561,0.103303,0.031733,0.055458,1.617249,-0.755124,1.617519,-0.755386,2234.631212
5,2275,159.000000,58.350000,0.043609,0.070371,0.096167,0.029888,0.053592,1.004570e-03,0.025825,0.096224,0.029930,0.053673,1.696389,-0.830169,1.696675,-0.830434,2259.460448
6,2300,140.550003,64.700001,0.032680,0.061529,0.089838,0.028142,0.051429,5.641274e-04,0.028324,0.089868,0.028163,0.051469,1.765012,-0.892459,1.765110,-0.892547,2284.289684
7,2325,122.849998,71.849998,0.021869,0.053202,0.084294,0.026417,0.048845,2.526202e-04,0.031116,0.084342,0.026449,0.048905,1.817399,-0.938404,1.817477,-0.938471,2309.118919
8,2350,106.050003,79.799999,0.011174,0.045438,0.079639,0.024725,0.045893,6.595018e-05,0.034191,0.079619,0.024712,0.045869,1.847036,-0.963813,1.847027,-0.963805,2333.948155
9,2375,90.200001,88.799999,0.000592,0.038240,0.075888,0.023021,0.042518,1.851502e-07,0.037647,0.075885,0.023020,0.042515,1.846861,-0.963664,1.846861,-0.963664,2358.777391
